# Project on Next product prediction

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import sys
import heapq
import seaborn as sns
import matplotlib
from numpy.core.multiarray import dtype
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model

import re
import pickle

import os
from pylab import rcParams
matplotlib.use('agg')
np.random.seed(42)

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.utils.np_utils import to_categorical
import re

In [3]:
sale_data_df=pd.read_csv("../input/new-dataset/Sales-Dataset-2-3w-3w.csv",encoding = "ISO-8859-1")
sale_data_df.head()

In [4]:
sale_data_df_combine=sale_data_df.replace(' ', '', regex=True)
sale_data_df_combine.head()

In [5]:

# reanming the DataFrame columns
sale_data_df_combine.rename(columns={'3-Words':'3-Words_combine','Next-Word':'Next-Word_combine'}, 
            inplace = True)
sale_data_df_combine

In [6]:
sale_data_result = pd.concat([sale_data_df, sale_data_df_combine], axis=1, join='inner')
display(sale_data_result)

In [7]:
brands = list(sale_data_result['3-Words_combine'])
  
# cleaning and combining words
all_brands_str = ' '.join(str(brand).lower() for brand in brands)
print(all_brands_str[0:100])

In [64]:
brands[0:5]

In [65]:
brands_next_word = list(sale_data_result['Next-Word'])
brands_next_word[0:5]

In [66]:
import pickle
tokenizer = Tokenizer()
tokenizer.fit_on_texts([all_brands_str])

pickle.dump(tokenizer, open('all_brands_3words_nextword_tokenizer.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([all_brands_str])[0]

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [67]:
#sequence_data

In [68]:
sequences = []

for i in range(1, len(sequence_data)):
    X_y_sequence_pair = sequence_data[i-1:i+1]
    sequences.append(X_y_sequence_pair)
    
sequences = np.array(sequences)

X = []
y = []

for i in sequences:
    X.append(i[0:1])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

In [69]:
X

In [70]:
y = to_categorical(y, num_classes = vocab_size)
print("X length=",len(X))
print("y length=",len(y))

In [71]:
X,y

In [72]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model
checkpoint = ModelCheckpoint("lstm_3input_next_word_model.h5", monitor='loss', verbose=1 ,save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

tensorboard_visualization = TensorBoard(log_dir='logs')

In [19]:
from tensorflow.keras.optimizers import Adam

model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=100, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_visualization])

In [8]:
model = load_model('../input/model-f/lstm_3input_next_word_model.h5')
tokenizer = pickle.load(open('../input/models/all_brands_tokenizer (1).pkl', 'rb'))

def predict_next_word(model, tokenizer, all_brands):
    brand_sequences = tokenizer.texts_to_sequences([all_brands])
    brand_sequences = np.array(brand_sequences)
    preds = np.argmax(model.predict(brand_sequences))
    predicted_word = ""
  
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
  
    print(predicted_word)
    return predicted_word

In [9]:
data = input("Enter your word: ")
next_word = predict_next_word(model, tokenizer, data)


In [15]:
output_list=sale_data_result[sale_data_result["3-Words_combine"]==data]["Next-Word_combine"].value_counts()

print(output_list.values)
len(output_list)
output_list.index

In [11]:
df_result=pd.DataFrame({"Items":list(output_list.index),"Counts":list(output_list.values)})
df_result

In [78]:
sale_data_result.head()

df_r=pd.DataFrame(output_list)
df_r
#df_r.index
#df_r.values

res_index=[]
for i in range(0,len(output_list)):
    res_index=output_list[i]
    print(res_index)
    #print(list(res_index))

if "".join(result['Next-Word'][0].split())=='BesivaBlackViscose':
    print(result['Next-Word'][0])

In [12]:
brand_index=sale_data_result['Next-Word'].value_counts().index
brand_index

result_final=[]
brand_list=[]
for i in range(len(result['Next-Word'])):
    if ("".join(result['Next-Word'][i].split()) in df_r.index):
        
        brand_list=result['Next-Word'][i]
        print(brand_list)
        #result_final.append(brand_list)
#result_final
#pd.DataFrame(result_final).value_counts()
        #print(result_final)

In [13]:
Item_index=df_result['Items'].value_counts().index
Item_index

In [ ]:
brand_list=[]
brand_list_c =[]
for i in range(len(brand_index)):
    brand_in="".join(brand_index[i].split())
    if (brand_in in output_list.index.tolist()):
        # print(brand_index[i],output_list.values[i])
        brand_list.append(brand_index[i])
        brand_list_c.append(output_list.values[i])
pd.DataFrame({"Brand" : brand_list,"Count": brand_list_c})

In [19]:
brand_list=[]
brand_list_c=[]
for i in range(len(brand_index)):
    brand_in="".join(brand_index[i].split())
    if (brand_in in output_list.index.tolist()):
        brand_list1=brand_index[i]
        brand_list.append(brand_list1)
        brand_list_c.append(output_list.values[i])
        #print(brand_list)

        #brand_list1=list(brand_list)
#brand_list1

df_f_out=pd.DataFrame({"brand_names":brand_list})
df_f_out                  
Item_brand_index=df_f_out['brand_names'].value_counts().index
Item_brand_index

In [ ]:
col = 'ID'
cols_to_replace = ['Next-Word_combine']
df3.loc[df3[col].isin(df1[col]), cols_to_replace] = df1.loc[df1[col].isin(df3[col]),cols_to_replace].values

In [99]:
result_final_out=[]

brand_list_out=[]
for i in range(len(df_f_out["brand_names"])): 
    if ("".join(df_f_out["brand_names"][i].split()) in Item_index):
        
        brand_list_out=df_f_out["brand_names"][i]
        #df_f_out.loc[:, ['brand_names']] = df_result[['Items']]
        #df_final=df_f_out.replace(df_f_out,df_result)
    #t.append(df_r.values)
    
    #print(brand_list_out)
    result_final_out.append(brand_list_out)
#t=df_r.values
#print(t)
result_final_out 
df_final
pd.DataFrame(result_final_out).value_counts()

In [124]:
result_final_out=[]
l1=[]
m1=[]
brand_list_out=[]
for i in range(len(df_f_out["brand_names"])): 
    if ("".join(df_f_out["brand_names"][i].split()) in Item_index):
        m=df_f_out["brand_names"][i]
        m1.append(m)
        for j in range(len(df_result["Counts"])):
            l=df_result["Counts"][j]
            l1.append(l)
print(l1)
print(m1)
#pd.DataFrame(m1,l1)
        

In [105]:
t=[]
for i in range(len(df_f_out["brand_names"])):
    t.append("".join(df_f_out["brand_names"][i].split()))
print(t)

In [44]:
result_final=[]
brand_list1=[]
for i in range(len(result['Next-Word'])): 
    if ("".join(result['Next-Word'][i].split()) in df_r.index):

        brand_list1=result['Next-Word'][i]
        #print(brand_list1)
    result_final.append(brand_list1)
result_final 
pd.DataFrame(result_final).value_counts() 
#print(result_final)

result_final=[] 
#brand_list=[] 
for i in range(len(result['Next-Word'])): 
    if ("".join(result['Next-Word'][i].split()) in df_r.index):

        brand_list=result['Next-Word'][i]
        print(brand_list)
    #result_final.append(brand_list)